# Connect to Milvus

In [5]:
from pymilvus import(
    Milvus,
    IndexType,
    Status,
    connections,
    FieldSchema,
    DataType,
    Collection,
    CollectionSchema
)

host = '158.175.181.136'
port = '8080'
password = '4XYg2XK6sMU4UuBEjHq4EhYE8mSFO3Qq'
user = 'root'
server_pem_path =  './data/cert.pem'

connections.connect(alias = 'default',
                host = host,
                port = port,
                user = user,
                password = password,
                server_pem_path=server_pem_path,
                server_name = 'localhost',
                secure = True)

In [84]:
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True), # Primary key
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=300,),
    FieldSchema(name="article", dtype=DataType.VARCHAR, max_length=2500,),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1024,),
    FieldSchema(name="school_year", dtype=DataType.INT16, max_length=300,),
    FieldSchema(name="in_effect", dtype=DataType.VARCHAR, max_length=100,),
    FieldSchema(name="file_links", dtype=DataType.VARCHAR, max_length=300),
    FieldSchema(name="created_at", dtype=DataType.VARCHAR, max_length=200,),
    FieldSchema(name="updated_at", dtype=DataType.VARCHAR, max_length=200,),
]

schema = CollectionSchema(fields, "news schema")

In [85]:
collection_names = ['recruitment', 'timetable', 'scholarship', 'academic_affairs', 'events']

In [86]:
for col in collection_names:
    collection = Collection(col, schema)
    index_params = {
        'metric_type':'L2',
        'index_type':"IVF_FLAT",
        'params':{"nlist":2048}
    }
    collection.create_index(field_name='embedding', index_params=index_params)

In [125]:
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True), # Primary key
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=300,),
    FieldSchema(name="article", dtype=DataType.VARCHAR, max_length=5000,),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1024,),
    FieldSchema(name='page_number', dtype=DataType.INT64),
    FieldSchema(name="school_year", dtype=DataType.INT16,),
    FieldSchema(name="in_effect", dtype=DataType.VARCHAR, max_length=100,),
    FieldSchema(name="file_links", dtype=DataType.VARCHAR, max_length=300),
    FieldSchema(name="created_at", dtype=DataType.VARCHAR, max_length=200,),
    FieldSchema(name="updated_at", dtype=DataType.VARCHAR, max_length=200,),
]

schema = CollectionSchema(fields, "news schema")

collection = Collection('student_handbook', schema)

index_params = {
    'metric_type':'L2',
    'index_type':"IVF_FLAT",
    'params':{"nlist":2048}
}
collection.create_index(field_name='embedding', index_params=index_params)

Status(code=0, message=)

In [126]:
connections._fetch_handler('default').list_collections()

['recruitment',
 'scholarship',
 'timetable',
 'student_handbook',
 'academic_affairs',
 'events']

# Process data

In [127]:
from glob import glob

paths = glob('./data/thesis_rag_data/*')

***

In [136]:
path = paths[6]
path

IndexError: list index out of range

In [133]:
import json
with open(path, 'r', encoding='utf-8') as rstream:
    data = json.load(rstream)

for d in data:
    d.pop('_id', None)
    d['school_year'] = int(d['school_year'])

In [134]:
collection = Collection('timetable')

In [135]:
collection.insert(data)

(insert count: 8, delete count: 0, upsert count: 0, timestamp: 452156547920035842, success count: 8, err count: 0

***

In [80]:
# for c in connections._fetch_handler('default').list_collections():
#     Collection(c).drop()

***

In [138]:
import os
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames as EmbedParams
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes
from ibm_watsonx_ai.foundation_models import Embeddings
from dotenv import load_dotenv

In [139]:
load_dotenv('./data/.env')
my_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": os.environ['WATSONX_APIKEY'],
}

# model_id = 'sentence-transformers/all-minilm-l12-v2'
model_id = 'intfloat/multilingual-e5-large'
gen_parms = None
project_id = os.environ['WATSONX_PROJECT_ID']
space_id = None
verify = False

# Set the truncate_input_tokens to a value that is equal to or less than the maximum allowed tokens for the embedding model that you are using. If you don't specify this value and the input has more tokens than the model can process, an error is generated.

embed_params = {
    EmbedParams.TRUNCATE_INPUT_TOKENS: 512,
}

model = Embeddings(
    model_id=model_id,
    credentials=my_credentials,
    params=embed_params,
    project_id=project_id,
    verify=verify
)

In [140]:
query="Khoa công nghệ thông tin là gì"

In [144]:
query_embeddings = model.embed_query(query)

col_name = "student_handbook"
collection = Collection(col_name)
collection.load()
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 5}
}
top_k = 3
output_fields = ['title', 'article'] #Section field to be added
results = collection.search(
    data=[query_embeddings],
    anns_field="embedding",
    param=search_params,
    limit=top_k,
    expr=None,
    output_fields=output_fields
)

In [167]:
for r in results[0]:
    print(r.distance)

0.2861247956752777
0.29169338941574097
0.29555103182792664
